In [1]:
import pandas as pd

In [2]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [3]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [4]:
dados.shape

(10000, 4)

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print("Treinaremos com %d elementos e testaremos com %d elementos." % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos.


In [6]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)

acuracia = dummy_stratified.score(teste_x, teste_y) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


In [7]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("Acurácia: %.2f%%" % acuracia)

Acurácia: 71.92%


In [8]:
from sklearn.model_selection import cross_validate

In [9]:
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score = False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation 3 = [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100 ))

Accuracy com cross validation 3 = [74.99 76.57]


In [10]:
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score = False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation 5 = [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100 ))

Accuracy com cross validation 5 = [75.21 76.35]


In [11]:
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score = False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation 10 = [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100 ))

Accuracy com cross validation 10 = [74.24 77.32]


### Kfold com aleatorização

In [12]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy média: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100 ))

In [13]:
from sklearn.model_selection import KFold

In [14]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy média: 75.78
Accuracy intervalo: [74.37 77.19]


In [15]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy média: 75.76
Accuracy intervalo: [73.26 78.26]


### Estratificação com validação cruzada

##### Proporção de exemplos desbalanceados entre as classes

In [16]:
dados_azar = dados.sort_values("vendido", ascending = True)
x_azar = dados_azar[["preco","idade_do_modelo","km_por_ano"]]
y_azar = dados_azar[["vendido"]]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [17]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy média: 57.84
Accuracy intervalo: [34.29 81.39]


In [18]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy média: 75.78
Accuracy intervalo: [72.30 79.26]


In [19]:
from sklearn.model_selection import StratifiedKFold

In [20]:
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy média: 75.78
Accuracy intervalo: [73.55 78.01]


### Gerando dados aleatórios

In [21]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2, 3, size = 10000)
dados.head(10)

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3
5,55405.26,1,11,23594.53374,11
6,93415.61,1,16,16077.30660,18
7,65265.09,0,10,18666.73466,12
8,43917.53,1,19,12607.56956,17
9,107860.41,1,15,24000.08742,13


In [22]:
dados.modelo_aleatorio.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1], dtype=int64)

corrigir o valor "-1"

In [23]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head(10)

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5
5,55405.26,1,11,23594.53374,13
6,93415.61,1,16,16077.30660,20
7,65265.09,0,10,18666.73466,14
8,43917.53,1,19,12607.56956,19
9,107860.41,1,15,24000.08742,15


In [24]:
dados.modelo_aleatorio.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1], dtype=int64)

In [25]:
dados['modelo'] = dados['modelo_aleatorio']
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio,modelo
0,30941.02,1,18,35085.22134,18,18
1,40557.96,1,20,12622.05362,24,24
2,89627.50,0,12,11440.79806,14,14
3,95276.14,0,3,43167.32682,6,6
4,117384.68,1,4,12770.11290,5,5


In [26]:
dados = dados.drop(columns = ['modelo_aleatorio'])
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [27]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1], dtype=int64)

### Validação cruzada usando grupos

In [28]:
from sklearn.model_selection import GroupKFold

In [30]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score = False)
imprime_resultados(results)

Accuracy média: 75.78
Accuracy intervalo: [73.67 77.90]
